<a href="https://colab.research.google.com/github/akansh12/CS6910_Assignment/blob/main/partB/Part_B_Akansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torchvision
import os
import torch.nn as nn
import matplotlib.pyplot as plt
import copy
from tqdm.notebook import tqdm
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = "cpu"

In [2]:
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
path2data_train="/content/drive/MyDrive/Nature_data/inaturalist_12K/train"
path2data_test = "/content/drive/MyDrive/Nature_data/inaturalist_12K/val"

In [3]:
train_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.RandomRotation((-20,20)),
transforms.RandomHorizontalFlip(p=0.5),
transforms.RandomVerticalFlip(p=0.5),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [4]:
data = datasets.ImageFolder(path2data_train, train_transforms)
test_data = datasets.ImageFolder(path2data_test, test_transforms)
n_val = int(np.floor(0.1 * len(data)))
n_train = len(data) - n_val
train_ds, val_ds = random_split(data, [n_train, n_val])

print("Number of datapoints in train: ", len(train_ds))
print("Number of datapoints in val: ", len(val_ds))
print("Number of datapoints in test: ", len(test_data))

Number of datapoints in train:  9000
Number of datapoints in val:  999
Number of datapoints in test:  2000


In [5]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False)
test_dl = DataLoader(test_data, batch_size=64, shuffle=False)

### Question 1

I will be experimenting with 
- InceptionV3, 
- DenseNet121
- ResNet50

Others can be added to the code.

In [6]:
def model_define(name= "resnet50", num_classes = 10, pretraining = True):
  if name == "resnet50":
    model = models.resnet50(pretrained=pretraining)
    model.fc = nn.Linear(2048, num_classes)
  if name == "inceptionv3":
    model = models.inception_v3(pretrained = pretraining)
    model.fc = nn.Linear(2048, 10)
  if name == "densenet121":
    model = models.densenet121(pretrained=pretraining)
    model.classifier = nn.Linear(1024, 10)
  return model.to(device)

In [7]:
model = model_define("resnet50");

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [8]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Answer

1. The dimensions of the images in your data may not be the same as that in the ImageNet data. How will you address this?
- ANS: The image size of ImageNET dataset is (224, 224, 3), while peforming augmentation, I will add Resize transform layer in my code to change all the input image of iNaturalist to 224,224,3

2. ImageNet has 1000 classes and hence the last layer of the pre-trained model would have 1000 nodes. However, the naturalist dataset has only 10 classes. How will you address this?

- ANS: I have changed the last fully connected layer with neurons equal to the number of classes, in this case it is 10. 

### Question 2

-ANSWER: 

Freezing is the word that is blanked in the question. I will be using the following methods to fine tune:
- Freezing of CONV layers and fine tuning only FC layers.
- Freezing of first few(10) conv layers and tuning the rest of the model.
- Fine tuning only conv layers while FC layer is freezed. 

### Question 3

In [9]:
#Loss Function
loss_func = nn.CrossEntropyLoss()
#Optimizer
from torch import optim
opt = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
### Helper functions
def metrics_b(out, y_true):
  pred = out.argmax(dim=1, keepdim=True)
  true_pred=pred.eq(y_true.view_as(pred)).sum().item()
  return true_pred

def loss_batch(loss_func, out, y_true, opt=None):
  loss = loss_func(out, y_true)
  metric_batch = metrics_b(out,y_true)
  if opt is not None:
    opt.zero_grad()
    loss.backward()
    opt.step()
  return loss.item(), metric_batch

def loss_epoch(model,loss_func,data_loader,opt=None):
  run_loss=0.0
  running_metric=0.0
  len_data=len(data_loader.dataset)
  for x, y in tqdm(data_loader):
    x=x.to(device)
    y=y.to(device)
    output=model(x)
    loss_b,metric_b=loss_batch(loss_func, output, y, opt)
    run_loss+=loss_b

    if metric_b is not None:
      running_metric+=metric_b

    

  loss=run_loss/float(len_data)
  metric=running_metric/float(len_data)

  return loss, metric

def train_val(model, params):
  num_epochs=params["num_epochs"]
  loss_func=params["loss_func"]
  opt=params["optimizer"]
  train_dl=params["train_dl"]
  val_dl=params["val_dl"]
  path2weights=params["path2weights"]

  loss_hist={
    "train": [],
    "val": [],
    }

  metric_hist={
    "train": [],
    "val": [],
    }

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss=float('inf')

  for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch, num_epochs- 1))
    model.train()
    train_loss,train_metric=loss_epoch(model,loss_func,train_dl,opt)
    loss_hist["train"].append(train_loss)
    metric_hist["train"].append(train_metric)

    model.eval()
    with torch.no_grad():
      val_loss,val_metric = loss_epoch(model,loss_func,val_dl)
      loss_hist["val"].append(val_loss)
      metric_hist["val"].append(val_metric)

    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      torch.save(model.state_dict(), path2weights)
      print("Copied best model weights!")


    print("train loss: %.6f, dev loss: %.6f, accuracy: %.2f"%(train_loss,val_loss,100*val_metric))

    model.load_state_dict(best_model_wts)
  return model, loss_hist, metric_hist

In [22]:
train_params = {"num_epochs": 30,
                "loss_func": loss_func,
                "train_dl":train_dl ,
                "val_dl":val_dl, 
                "test_dl": test_dl,
                "path2weights": "./best_model.pt",
                "optimizer": opt,
                "loss_func": loss_func,
                "model": model_define("resnet50")
               }

In [ ]:
for param in model.parameters():
    param.require_grad = True

model_trained,loss_hist,metric_hist=train_val(model,train_params)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0/29


  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Copied best model weights!
train loss: 0.229351, dev loss: 0.042235, accuracy: 10.71
Epoch 1/29


...

train loss: 0.099244, dev loss: 0.046120, accuracy: 13.91
Epoch 5/29


  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

train loss: 0.098662, dev loss: 0.054697, accuracy: 9.31
Epoch 6/29


  0%|          | 0/282 [00:00<?, ?it/s]

In [11]:
def accuracy(y_pred,y_true):
    y_pred = torch.exp(y_pred)
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

for param in model.parameters():
    param.require_grad = True
model = model.to(device)

schedular = optim.lr_scheduler.ReduceLROnPlateau(opt,factor = 0.1,patience = 5)
epochs = 5
valid_loss_min = np.Inf

In [13]:
device

'cpu'

In [12]:
train_loss_hist = []
valid_loss_hist = []
train_acc_hist = []
valid_acc_hist = []


for i in range(epochs):
    
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0.0
    valid_acc = 0.0 
    
    
    model.train()
    
    for images,labels in tqdm(train_dl):
        
        images = images.to(device)
        labels = labels.to(device)
        
        ps = model(images)
        loss = loss_func(ps,labels)
        
        opt.zero_grad()
        loss.backward()
        optim.step()
        
        train_acc += accuracy(ps,labels)
        train_loss += loss.item()
        
    avg_train_acc = train_acc / len(train_dl)
    train_acc_hist.append(avg_train_acc)
    avg_train_loss = train_loss / len(train_dl)
    train_loss_hist.append(avg_train_loss)
        
    model.eval()
    with torch.no_grad():
        
        for images,labels in tqdm(test_dl):
            
            images = images.to(device)
            labels = labels.to(device)
            
            ps = model(images)
            loss = loss_func(ps,labels)
            
            valid_acc += accuracy(ps,labels)
            valid_loss += loss.item()
            
            
        avg_valid_acc = valid_acc / len(test_dl)
        valid_acc_hist.append(avg_valid_acc)
        avg_valid_loss = valid_loss / len(test_dl)
        valid_loss_hist.append(avg_valid_loss)
        
        schedular.step(avg_valid_loss)
        
        if avg_valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).   Saving model ...'.format(valid_loss_min,avg_valid_loss))
            torch.save({
                'epoch' : i,
                'model_state_dict' : model.state_dict(),
                'optimizer_state_dict' : opt.state_dict(),
                'valid_loss_min' : avg_valid_loss
            },'Resnet50_100_epochs.pt')
            
            valid_loss_min = avg_valid_loss
            
            
    print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1,avg_train_loss,avg_train_acc))
    print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1,avg_valid_loss,avg_valid_acc))

  0%|          | 0/282 [00:00<?, ?it/s]

AttributeError: ignored